## API - LINXMOVIMENTO - CARGA DE DADOS - ORACLE

##### coding: utf-8
##### @Tiago de Camargo::
### github - https://github.com/ticamargo
### linkedin - https://www.linkedin.com/in/tiagodecamargo/

In [ ]:
## IMPORTACAO DE PACOTES
from unicodedata import normalize
from datetime import datetime
import pandas as pd
import cx_Oracle
import requests
import warnings

In [ ]:
## MOSTRAR COLUNAS DO DATAFRAME! FACILITA A VISUALIZACAO
pd.set_option('display.max_columns', 80)
warnings.simplefilter('ignore')

In [ ]:
## SELECIONAR O ARQUIVO PARA CARGA DE DADOS, NESTE CASO UM CSV
dataframe = pd.read_csv('exemplo_dados.csv', sep=',', low_memory=False, dtype=str)

In [ ]:
## CRIAR UM NOVO DATAFRAME DO DATAFRAME COM TODAS AS COLUNAS, SELECIONANDO APENAS AS COLUNAS QUE SERÃO UTILIZADAS
df = dataframe[['portal','cnpj_emp','transacao','documento','chave_nf','ecf','numero_serie_ecf','modelo_nf','data_documento','data_lancamento','codigo_cliente','serie','desc_cfop','id_cfop','cod_vendedor','quantidade','preco_custo','valor_liquido','desconto','valor_total','total_dinheiro','total_cheque','total_cartao','total_crediario','total_convenio','operacao','tipo_transacao','cod_produto','cod_barra','cancelado','excluido','soma_relatorio','identificador','preco_unitario','hora_lancamento','natureza_operacao','cod_sefaz_situacao','desc_sefaz_situacao','protocolo_aut_nfe','total_cheque_prazo','cod_natureza_operacao','preco_tabela_epoca','desconto_total_item']]

In [ ]:
## CCONTAGEM DO DATAFRAME
df.count()

In [ ]:
# REMOVER, SE HOUVER TRAÇOS NEGATIVOS EM VALOR_TOTAL
df.loc[0:, 'valor_total'] = df['valor_total'].str.replace('-', '')

In [ ]:
## LIMPAR CARACTERES DA COLUNA COD_NATUREZA_OPERACAO
df.loc[0:, 'cod_natureza_operacao'] = df['cod_natureza_operacao'].str.strip()

In [ ]:
## CARREGAR UMA LISTA COM O INDEX ONDE NA COLUNA COD_NATUREZA_OPERACAO = 1.201 - VALOR_TOTAL SERÁ NEGATIVO 
lista_devolucao = []
for row in df.itertuples():
    if row.cod_natureza_operacao == '1.201':
        lista_devolucao.append(row.Index)

In [ ]:
## TRANSFORMAR EM NEGATIVO VALOR_TOTAL E QUANTIDADE DA TUPLA LISTA_DEVOLUCAO
df.loc[lista_devolucao, 'valor_total'] = df['valor_total'].map(lambda x: "-{}".format(x))
df.loc[lista_devolucao, 'quantidade'] = df['quantidade'].map(lambda x: "-{}".format(x))

In [ ]:
## PREENCHER VALORES NAN IDENTIFICADOS NO CONSUMO DA API
values = {'chave_nf': ' ', 'numero_serie_ecf':' ', 'modelo_nf': '0','desc_cfop': ' ', 'id_cfop': '0', 'valor_total': '0', 'cod_barra': '0', 'cod_sefaz_situacao': '0', 'desc_sefaz_situacao': ' ', 'protocolo_aut_nfe': '0', 'desconto_total_item': '0'}
df = df.fillna(value=values)

In [ ]:
## RETIRANDO ACENTOS PARA TABELA ASCCI NAS COLUNAS
df.loc[0:, 'desc_cfop'] = df['desc_cfop'].apply(lambda x: normalize('NFKD', x).encode('ascii', 'ignore').decode('ascii'))
df.loc[0:, 'natureza_operacao'] = df['natureza_operacao'].apply(lambda x: normalize('NFKD', x).encode('ascii', 'ignore').decode('ascii'))

In [ ]:
## FORMATAR DATAS PARA PADRAO CRIADO NA BASE DE DADOS
df.loc[0:, 'data_documento'] = df['data_documento'].str.replace('-', '')
df.loc[0:, 'data_lancamento'] = df['data_lancamento'].str.replace('-', '')

In [ ]:
## FORMATAR DATAS PARA PADRAO CRIADO NA BASE DE DADOS
df.loc[0:, 'data_documento'] = df['data_documento'].map(lambda d: "{}".format(d[0:8]))
df.loc[0:, 'data_lancamento'] = df['data_lancamento'].map(lambda d: "{}".format(d[0:8]))

In [ ]:
## CRIAR LISTA DE COLUNAS QUE DEVEM SER DO TIPO INT
cols_int = ['portal','cnpj_emp','transacao','documento','ecf','modelo_nf','codigo_cliente','id_cfop','cod_vendedor','cod_produto','cod_barra','cod_sefaz_situacao','protocolo_aut_nfe']

In [ ]:
## CONVERTER PARA INT LISTA DE COLUNAS QUE DEVEM SER DO TIPO INT
df[cols_int] = df[cols_int].apply(lambda x: pd.to_numeric(x.astype('int64'), errors='coerce'))

In [ ]:
## PERCORRER COLUNA QUANTIDADE, PRIMEIRO FLOAT, DEPOIS ROUND() DEPOIS INT
df['quantidade'] = df['quantidade'].astype('float64', errors='ignore')
df['quantidade'] = df['quantidade'].round()
df['quantidade'] = df['quantidade'].astype('int64')

In [ ]:
## CRIAR LISTA DE COLUNAS COLUNAS QUE DEVEM SER DO TIPO FLOAT
cols_float = ['preco_custo','valor_liquido','desconto','valor_total','total_dinheiro','total_cheque','total_cartao','total_crediario','total_convenio','preco_unitario','total_cheque_prazo','preco_tabela_epoca','desconto_total_item']

In [ ]:
## VERIFICAR POR DADOS NAN NO DATAFRAME
df.isnull().sum() != 0

In [ ]:
## VERIFICAR TIPOS DE DADOS DAS COLUNAS
df.apply(lambda x: pd.api.types.infer_dtype(x.values))

In [ ]:
## CONVERTER PARA FLOAT COLUNAS DE PREÇOS
df[cols_float] = df[cols_float].apply(lambda x: pd.to_numeric(x.astype(str), errors='coerce'))

In [ ]:
## MOSTRA O DATAFRAME
df.head()

In [ ]:
## CRIA UMA LISTA
df_list = []

In [ ]:
## LISTA CRIADA RECEBE DADOS DO DATAFRAME
df_list = df.values.tolist()

In [ ]:
## MOSTRA PRIMEIRO ITEM DA LISTA
df_list[lista_devolucao[0]]

In [ ]:
## MOSTRA TAMANHO DA LISTA
print(len(df_list))

In [ ]:
## STRING DE CONEXÃO ORACLE
connect = cx_Oracle.connect('')

In [ ]:
## INSERIR NA BASE DE DADOS - METODO MATRIZ
rows = df_list
cursor = connect.cursor()
cursor.bindarraysize = 43
cursor.setinputsizes(int,int,int,int,200,int,50,int,8,8,int,50,300,int,int,int,float,float,float,float,\
                     float,float,float,float,float,2,1,int,int,1,1,1,50,float,8,50,int,50,int,float,15,float,float)
statement = 'INSERT INTO DW_LINX (portal,cnpj_emp,transacao,documento,chave_nf,ecf,numero_serie_ecf,\
modelo_nf,data_documento,data_lancamento,codigo_cliente,serie,desc_cfop,id_cfop,cod_vendedor,quantidade,\
preco_custo,valor_liquido,desconto,valor_total,total_dinheiro,total_cheque,total_cartao,total_crediario,\
total_convenio,operacao,tipo_transacao,cod_produto,cod_barra,cancelado,excluido,soma_relatorio,identificador,\
preco_unitario,hora_lancamento,natureza_operacao,cod_sefaz_situaca,desc_sefaz_situacao,protocolo_aut_nfe,\
total_cheque_prazo,cod_natureza_operacao,preco_tabela_epoca,desconto_total_item) \
VALUES (:1,:2,:3,:4,:5,:6,:7,:8,:9,:10,:11,:12,:13,:14,:15,:16,:17,:18,:19,:20,:21,:22,:23,:24,\
:25,:26,:27,:28,:29,:30,:31,:32,:33,:34,:35,:36,:37,:38,:39,:40,:41,:42,:43)'
for i in range(0, len(rows), 10000):
    try:
        cursor.executemany(statement, rows[i:i+10000])
        connect.commit()
        print('Range :::', i ,'Até:::', i+9999 ,'::: INSERIDO/COMMIT COM SUCESSO :::')
    except cx_Oracle.DatabaseError as e:
        errorObj, = e.args
        print("NO RANGE ", i ,"A LINHA ", cursor.rowcount, "ESTÁ COM ERRO ", errorObj.message)
        connect.rollback()
        cursor.close()
        connect.close()
        break

cursor.close()
connect.close()
print('Tudo OK!!')